In [17]:
# ベイズ推定による回帰分析

require 'daru'
require 'numo/narray'
require 'open3'
require 'nyaplot'

DATASET_NUMS = [4, 5, 10, 100]

BETA = 1.0 / (0.3) ** 2 # 真の分布の分散
ALPHA = 1.0 / 100 ** 2  # 事前分布の分散
ORDER = 9               # 多項式の字数

def normal_rand(mu = 0, sigma = 1.0)
  random = Random.new
  (Math.sqrt(-2 * Math.log(random.rand)) * Math.sin(2 * Math::PI * random.rand) * sigma) + mu
end

# データセット {x_n,y_n} (n=1...N) を用意
def create_dataset(num)
  dataset = Daru::DataFrame.new({'x': [], 'y': []})  

  num.times do |i|
    x = i.to_f / (num - 1).to_f
    y = Math.sin(2 * Math::PI * x) + normal_rand(0, 0.3)
    dataset.add_row(Daru::Vector.new([x, y], index: [:x, :y]))
  end
  
  return dataset
end

# 事後分布に基づく推定曲線、および、事後分布の平均と分散を計算
def resolve(dataset, m)
  t = dataset.y

  columns = {}
  (m+1).times do |i|
    columns["x**#{i}"] = dataset.x ** i
  end
  phis = Daru::DataFrame.new(columns)

  phiphi = nil
  phis.each_row_with_index do |line, index|
    phi = Daru::DataFrame.new(x: line)
    if index == 0
      phiphi = phi.to_matrix * phi.transpose.to_matrix
    else
      phiphi += phi.to_matrix * phi.transpose.to_matrix
    end
  end

  s_inv = Matrix[*(ALPHA * Numo::DFloat.eye(m + 1))] + BETA * phiphi
  s = s_inv.inv # 事後分布の共分散行列

  # 平均 m(x)
  mean_fun = lambda {|x0|
    phi_x0 = Numo::NArray[*((m + 1).times.map {|i| (x0 ** i).to_a })]
    tmp = 0
    phis.each_row_with_index do |line, index|
      if index == 0
        tmp = t[index] * Numo::NArray[*line.to_a]
        next
      end

      tmp += t[index] * Numo::NArray[*line.to_a]
    end
    BETA * phi_x0.transpose.dot(Numo::NArray[*s.to_a]).dot(tmp)
  }

  # 標準偏差 s(x)
  deviation_fun = lambda {|x0|
    phi_x0 = Numo::NArray[*((m + 1).times.map {|i| (x0 ** i).to_a })]
    deviation = (1.0 / BETA + phi_x0.transpose.dot(Numo::NArray[*s.to_a]).dot(phi_x0)).map {|v| v < 0 ? Float::NAN : Math.sqrt(v) }
    deviation.diagonal
  }

  tmp = nil
  phis.each_row_with_index do |line, index|
    if index == 0
      tmp = t[index] * Numo::NArray[*line.to_a]
      next
    end

    tmp += t[index] * Numo::NArray[*line.to_a]
  end
  mean = BETA * Numo::NArray[*s.to_a].dot(tmp).flatten

  return mean_fun, deviation_fun, mean, s
end

fig1 = Nyaplot::Frame.new
fig2 = Nyaplot::Frame.new

DATASET_NUMS.each do |num|
  train_set = create_dataset(num)
  mean_fun, deviation_fun, mean, sigma = resolve(train_set, ORDER)
  command = "python -c 'import numpy; print numpy.random.multivariate_normal(#{mean.to_a.inspect}, #{sigma.to_a.inspect}, 4).tolist()'"
  output, std_error, status = Open3.capture3(command)
  ws_samples = Daru::DataFrame.rows(eval(output))
  
  # トレーニングセットを表示
  plot1 = Nyaplot::Plot.new
  scatter1 = plot1.add(:scatter, train_set.x.to_a, train_set.y.to_a)
  scatter1.color('blue')
  scatter1.title('train_set')
  
  plot1.configure do
    x_label("N=#{num}")
    y_label('')
    xrange([-0.05, 1.05])
    yrange([-2, 2])
    legend(true)
    height(300)
    width(490)
  end
  
  plot2 = Nyaplot::Plot.new
  scatter2 = plot2.add(:scatter, train_set.x.to_a, train_set.y.to_a)
  scatter2.color('blue')
  scatter2.title('train_set')

  plot2.configure do
    x_label("N=#{num}")
    y_label('')
    xrange([-0.05, 1.05])
    yrange([-2, 2])
    legend(true)
    height(300)
    width(490)
  end
  
  linex = Numo::NArray[*(0..1).step(0.01).to_a]

  # 真の曲線を表示
  liney = (2 * Math::PI * linex).map {|v| Math.sin(v) }
  collect_line = plot1.add(:line, linex, liney)
  collect_line.color('green')
  collect_line.title('collect')

  # 平均と標準偏差の曲線を表示
  m = mean_fun.call(linex)
  d = deviation_fun.call(linex)
  mean_line = plot1.add(:line, linex, m)
  mean_line.color('red')
  mean_line.title('mean')
  lower_std_line = plot1.add(:line, linex, m - d)
  lower_std_line.color('black')
  lower_std_line.title('')
  upper_std_line = plot1.add(:line, linex, m + d)
  upper_std_line.color('black')
  upper_std_line.title('')

  # 多項式のサンプルを表示
  liney = m
  mean_line = plot2.add(:line, linex, liney)
  mean_line.color('red')
  mean_line.title('mean')
  
  f = lambda {|x, ws|
    y = 0
    ws.each_with_index do |w, i|
      y += w * (x ** i.to_i)
    end
    y
  }

  ws_samples.each_row do |ws|
    liney = f.call(linex, ws)
    sample_line = plot2.add(:line, linex, liney)
    sample_line.color('pink')
    sample_line.title('sample')
  end
  
  fig1.add(plot1)
  fig2.add(plot2)
end

fig1.show
fig2.show

<main>:9: warning: already initialized constant BETA
<main>:7: warning: previous definition of BETA was here
<main>:10: warning: already initialized constant ALPHA
<main>:8: warning: previous definition of ALPHA was here
<main>:11: warning: already initialized constant ORDER
<main>:9: warning: previous definition of ORDER was here


#<Nyaplot::Frame:0x007f29d5182b70 @properties={:panes=>[#<Nyaplot::Plot:0x007f29d50950f0 @properties={:diagrams=>[#<Nyaplot::Diagram:0x007f29d508bd20 @properties={:type=>:scatter, :options=>{:x=>"data0", :y=>"data1", :color=>"blue", :title=>"train_set"}, :data=>"cf23df21-c133-4d61-a893-56c92b51968d"}, @xrange=[0.0, 1.0], @yrange=[-1.0118047685605274, 1.4292117828295994]>, #<Nyaplot::Diagram:0x007f29d506d438 @properties={:type=>:line, :options=>{:x=>"data0", :y=>"data1", :color=>"green", :title=>"collect"}, :data=>"aa013aea-d024-4b3e-ad75-0f48ce8c3f3f"}, @xrange=[0.0, 1.0], @yrange=[-1.0, 1.0]>, #<Nyaplot::Diagram:0x007f29d50312a8 @properties={:type=>:line, :options=>{:x=>"data0", :y=>"data1", :color=>"red", :title=>"mean"}, :data=>"0680c154-c30b-4347-9aef-837126d90527"}, @xrange=[0.0, 1.0], @yrange=[-2.8568648818896016, 1.4400792753215892]>, #<Nyaplot::Diagram:0x007f29d501e6d0 @properties={:type=>:line, :options=>{:x=>"data0", :y=>"data1", :color=>"black", :title=>""}, :data=>"04eb7a22-c11a-4b36-80d5-a7347cd1cbf9"}, @xrange=[0.0, 1.0], @yrange=[-10.091512245351083, 1.005359177680357]>, #<Nyaplot::Diagram:0x007f29d4fd1dd0 @properties={:type=>:line, :options=>{:x=>"data0", :y=>"data1", :color=>"black", :title=>""}, :data=>"8f8fae6f-c3b6-4206-80b0-4a7f97ead4d4"}, @xrange=[0.0, 1.0], @yrange=[-0.6181566750538654, 4.692283215129504]>], :options=>{:x_label=>"N=4", :y_label=>"", :xrange=>[-0.05, 1.05], :yrange=>[-2, 2], :legend=>true, :height=>300, :width=>490, :zoom=>true}}>, #<Nyaplot::Plot:0x007f29d46e02d0 @properties={:diagrams=>[#<Nyaplot::Diagram:0x007f29d46b3190 @properties={:type=>:scatter, :options=>{:x=>"data0", :y=>"data1", :color=>"blue", :title=>"train_set"}, :data=>"8f3c769c-7416-4e2a-b837-9216fd5ef9ee"}, @xrange=[0.0, 1.0], @yrange=[-1.5496532576163118, 0.9580265787255592]>, #<Nyaplot::Diagram:0x007f29d466da00 @properties={:type=>:line, :options=>{:x=>"data0", :y=>"data1", :color=>"green", :title=>"collect"}, :data=>"40e1af83-1a16-41cb-aaf2-b56f09f85af5"}, @xrange=[0.0, 1.0], @yrange=[-1.0, 1.0]>, #<Nyaplot::Diagram:0x007f29d5ed0138 @properties={:type=>:line, :options=>{:x=>"data0", :y=>"data1", :color=>"red", :title=>"mean"}, :data=>"82df7b9a-3e15-438e-83eb-3a508b17619d"}, @xrange=[0.0, 1.0], @yrange=[-1.7647568476762052, 0.9556425939925632]>, #<Nyaplot::Diagram:0x007f29d6001368 @properties={:type=>:line, :options=>{:x=>"data0", :y=>"data1", :color=>"black", :title=>""}, :data=>"dacd6959-bdf8-4100-bc48-c491d96ecdfb"}, @xrange=[0.0, 1.0], @yrange=[-3.2524265314447183, 0.531814524992134]>, #<Nyaplot::Diagram:0x007f29d5b02588 @properties={:type=>:line, :options=>{:x=>"data0", :y=>"data1", :color=>"black", :title=>""}, :data=>"df233dfd-d8db-48af-85a4-e4017caadc55"}, @xrange=[0.0, 1.0], @yrange=[-1.1528688284752329, 1.3857829166587567]>], :options=>{:x_label=>"N=5", :y_label=>"", :xrange=>[-0.05, 1.05], :yrange=>[-2, 2], :legend=>true, :height=>300, :width=>490, :zoom=>true}}>, #<Nyaplot::Plot:0x007f29d602cf90 @properties={:diagrams=>[#<Nyaplot::Diagram:0x007f29d603bb30 @properties={:type=>:scatter, :options=>{:x=>"data0", :y=>"data1", :color=>"blue", :title=>"train_set"}, :data=>"c3ed9bf9-1d83-4b17-ab5d-60bb0fe3eb83"}, @xrange=[0.0, 1.0], @yrange=[-1.2830368745204372, 1.286983098724211]>, #<Nyaplot::Diagram:0x007f29d6066718 @properties={:type=>:line, :options=>{:x=>"data0", :y=>"data1", :color=>"green", :title=>"collect"}, :data=>"94140413-ee54-4590-bf67-a3326880fb37"}, @xrange=[0.0, 1.0], @yrange=[-1.0, 1.0]>, #<Nyaplot::Diagram:0x007f29d60da578 @properties={:type=>:line, :options=>{:x=>"data0", :y=>"data1", :color=>"red", :title=>"mean"}, :data=>"bc80255e-c969-45c6-aaca-8a60df6f1e13"}, @xrange=[0.0, 1.0], @yrange=[-1.3589407484351765, 0.8977299521472032]>, #<Nyaplot::Diagram:0x007f29d60ff850 @properties={:type=>:line, :options=>{:x=>"data0", :y=>"data1", :color=>"black", :title=>""}, :data=>"5e83224f-501a-4c44-a9b8-621ee26e558e"}, @xrange=[0.0, 1.0], @yrange=[-1.7315178164709388, 0.542651396303271]>, #<Nyaplot::Diagram:

#<Nyaplot::Frame:0x007f29d51829e0 @properties={:panes=>[#<Nyaplot::Plot:0x007f29d508a4e8 @properties={:diagrams=>[#<Nyaplot::Diagram:0x007f29d5088620 @properties={:type=>:scatter, :options=>{:x=>"data0", :y=>"data1", :color=>"blue", :title=>"train_set"}, :data=>"24f17138-53d4-450c-b93f-adbb5feee32d"}, @xrange=[0.0, 1.0], @yrange=[-1.0118047685605274, 1.4292117828295994]>, #<Nyaplot::Diagram:0x007f29d4fc1a70 @properties={:type=>:line, :options=>{:x=>"data0", :y=>"data1", :color=>"red", :title=>"mean"}, :data=>"1b88a782-17bd-45df-9450-50edb81a655b"}, @xrange=[0.0, 1.0], @yrange=[-2.8568648818896016, 1.4400792753215892]>, #<Nyaplot::Diagram:0x007f29d4fac788 @properties={:type=>:line, :options=>{:x=>"data0", :y=>"data1", :color=>"pink", :title=>"sample"}, :data=>"36f9c77b-e5a8-435e-a291-bfae9a39b5c1"}, @xrange=[0.0, 1.0], @yrange=[-1.405352977389694, 1.692106064355932]>, #<Nyaplot::Diagram:0x007f29d4f7b228 @properties={:type=>:line, :options=>{:x=>"data0", :y=>"data1", :color=>"pink", :title=>"sample"}, :data=>"22a0b460-b633-4a4a-b542-dfaa7400abd5"}, @xrange=[0.0, 1.0], @yrange=[-4.046806377466698, 2.2825116961411474]>, #<Nyaplot::Diagram:0x007f29d4f4f948 @properties={:type=>:line, :options=>{:x=>"data0", :y=>"data1", :color=>"pink", :title=>"sample"}, :data=>"6d5da45c-6121-41a5-875b-e9f9893ad6b5"}, @xrange=[0.0, 1.0], @yrange=[-2.1231683159479418, 4.288390748268249]>, #<Nyaplot::Diagram:0x007f29d4f24ec8 @properties={:type=>:line, :options=>{:x=>"data0", :y=>"data1", :color=>"pink", :title=>"sample"}, :data=>"6f83d6ff-7681-4b15-8791-e4daf78d41ff"}, @xrange=[0.0, 1.0], @yrange=[-7.042684266093417, 2.4143427635925825]>], :options=>{:x_label=>"N=4", :y_label=>"", :xrange=>[-0.05, 1.05], :yrange=>[-2, 2], :legend=>true, :height=>300, :width=>490, :zoom=>true}}>, #<Nyaplot::Plot:0x007f29d46b2cb8 @properties={:diagrams=>[#<Nyaplot::Diagram:0x007f29d46a3c68 @properties={:type=>:scatter, :options=>{:x=>"data0", :y=>"data1", :color=>"blue", :title=>"train_set"}, :data=>"5451cf9d-921b-496f-9ef6-2259603de2df"}, @xrange=[0.0, 1.0], @yrange=[-1.5496532576163118, 0.9580265787255592]>, #<Nyaplot::Diagram:0x007f29d5c27878 @properties={:type=>:line, :options=>{:x=>"data0", :y=>"data1", :color=>"red", :title=>"mean"}, :data=>"35a702fb-1388-4fb8-81e3-6cd6ac316011"}, @xrange=[0.0, 1.0], @yrange=[-1.7647568476762052, 0.9556425939925632]>, #<Nyaplot::Diagram:0x007f29d606dd38 @properties={:type=>:line, :options=>{:x=>"data0", :y=>"data1", :color=>"pink", :title=>"sample"}, :data=>"f196287b-b7e7-48b9-86e9-725ba8fcbecc"}, @xrange=[0.0, 1.0], @yrange=[-3.4585769995453512, 1.0120807156920684]>, #<Nyaplot::Diagram:0x007f29d60c01f0 @properties={:type=>:line, :options=>{:x=>"data0", :y=>"data1", :color=>"pink", :title=>"sample"}, :data=>"0a14555d-b13b-450d-a927-12a5e9f44477"}, @xrange=[0.0, 1.0], @yrange=[-1.861044467989701, 1.3288750861945946]>, #<Nyaplot::Diagram:0x007f29d5d826a0 @properties={:type=>:line, :options=>{:x=>"data0", :y=>"data1", :color=>"pink", :title=>"sample"}, :data=>"e07bda17-ef75-4464-8cfe-e9d5f1564f85"}, @xrange=[0.0, 1.0], @yrange=[-2.3039091629579245, 1.5119125074264128]>, #<Nyaplot::Diagram:0x007f29d5e20b70 @properties={:type=>:line, :options=>{:x=>"data0", :y=>"data1", :color=>"pink", :title=>"sample"}, :data=>"b29bcd89-32bb-4b7d-9e70-f5fadcae6c16"}, @xrange=[0.0, 1.0], @yrange=[-3.1796814566675633, 0.8398973238202814]>], :options=>{:x_label=>"N=5", :y_label=>"", :xrange=>[-0.05, 1.05], :yrange=>[-2, 2], :legend=>true, :height=>300, :width=>490, :zoom=>true}}>, #<Nyaplot::Plot:0x007f29d603b0e0 @properties={:diagrams=>[#<Nyaplot::Diagram:0x007f29d6039a10 @properties={:type=>:scatter, :options=>{:x=>"data0", :y=>"data1", :color=>"blue", :title=>"train_set"}, :data=>"e94aa699-8c94-47cd-871f-83f29e712755"}, @xrange=[0.0, 1.0], @yrange=[-1.2830368745204372, 1.286983098724211]>, #<Nyaplot::Diagram:0x007f29d5d15c08 @properties={:type=>:line, :options=>{:x=>"data0", :y=>"data1", :color=>"red", :title=>"mean"}, :data=>"6f19e589-9911-4e66-9abd-